In [21]:
import feedparser
import hashlib
from threading import Timer
import time
from pandas import DataFrame
import datetime
import pandas as pd
import os.path
import csv
from random import randint

In [22]:
def write_dump(path, data):
    keys = data[0].keys()
    with open(path, 'wb') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

In [23]:
def open_dump(path):
    if os.path.isfile(path):
        res = []
        with open(path) as csvfile:
            reader = csv.DictReader(csvfile)
            for r in reader:
                res.append(r)
            return res    

In [26]:
update_time = 60 # in seconds
dump_file = "news.csv"
set_columns = ['dt', 'ts', 'title', 'hash']

def scrap(hashes):
    ts = int(time.time())
#    dt =  pd.to_datetime(datetime.datetime.now())
    data_temp = []
    feed_raw = feedparser.parse('https://fivethirtyeight.com/features/feed/')
    feed_entries = feed_raw['entries']
    dt = datetime.datetime.now()
    for f in feed_entries:
        
        title = f['title'].encode('utf-8')
        hashed_title = hashlib.md5(title).hexdigest()
        dt = dt + datetime.timedelta(hours=randint(1,12))
        if not hashed_title in hashes:
            print "news! ", title
            entry = {'dt': dt, 'ts' : ts, 'title' :  title, 'hash': hashed_title} 
            data_temp.append(entry)

    #df = DataFrame.from_records(data_temp, columns = set_columns) 
    return data_temp

def run_dump():
    data = open_dump(dump_file)
    if data is None or data == []:
        print "No CSV dump"
        # first start without csv
        res = scrap([])
        write_dump(dump_file, res)
    else:
        hashes = [e['hash'] for e in data]
        res = scrap(hashes)
        data = data + res
        write_dump(dump_file, data)
    Timer(update_time, run_dump).start()
 

In [27]:
run_dump()      

No CSV dump
news!  Significant Digits For Wednesday, Jan. 25, 2017
news!  Chat: How’s Our Democracy Doing?
news!  Trump Finds The Weak Spot In Obama’s Protections For Scientists
news!  The Biggest Oscar Snubs And Surprises
news!  The Boston Bruins Look Good On Paper, So Why Do They Suck?
news!  How Our Oscars Tracker Works
news!  Significant Digits For Tuesday, Jan. 24, 2017
news!  Politics Podcast: The Beginning Of The Trump Presidency
news!  The Patriots Won With Less Talent Than Usual
news!  Three Days Into Trump’s Presidency, 45 Percent Of Americans Disapprove Of His Performance
news!  Matt Ryan Is No Fluke
news!  Higher Rates Of Hate Crimes Are Tied To Income Inequality
news!  Significant Digits For Monday, Jan. 23, 2017
news!  The Long March Ahead For Democrats
news!  The Invisible Undecided Voter
news!  It Wasn’t Clinton’s Election To Lose
news!  The Electoral College Blind Spot
news!  Trump’s Executive Order On Obamacare Means Everything And Does Nothing
news!  How Unconscious 

In [28]:
def read_dump(path):
    return pd.read_csv(path, index_col=0,parse_dates=[0])

In [37]:
def freq():
    df = read_dump(dump_file)
    group = df.groupby(pd.TimeGrouper(freq='1H'))

    group_counts = group.agg({'title':'count'})
    delta = group_counts.index.max() - group_counts.index.min()
    total = float(group_counts['title'].sum())
    total_hours =  delta.seconds / 60 / 60
    print total_hours
    if total_hours == 0:
        frequency = 1
    else: 
        frequency = total / total_hours
    return frequency



In [38]:
print "news pear hour: ", abs(freq())

news pear hour:  18
1.11111111111
